In [ ]:
import requests

def get_html(page:int, limit:int, book_id:str, review_type:str = "P"):
        
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'zh-CN,zh;q=0.9,zh-HK;q=0.8',
        'Referer': f'https://accounts.douban.com/',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'Cookie': 'bid=ayh_shqs3Gw; __utmv=30149280.20844; _vwo_uuid_v2=D1CE8AB8EA4487082BD1D1A2DE8302E94|0c05a290516b717927f2759b8bc4170e; __utmz=30149280.1744259183.6.6.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; _pk_id.100001.3ac3=1bce67d53826d441.1744619621.; __utma=81379588.39970239.1744678151.1744678151.1744678151.1; __utmz=81379588.1744678151.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); __utma=30149280.1376539290.1741879311.1744678151.1744777267.8; push_noty_num=0; push_doumail_num=0; ll="108288"; vtd-d="5"; viewed="3013597_30313225_37393112_36728010_35577790_35698970"; ap_v=0,6.0; _pk_ref.100001.3ac3=%5B%22%22%2C%22%22%2C1752628112%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3Dpxh7EoRhGUCUVfytsRPvPKa67grXPSHLPMIcSjuX2B59iu4qnFrMs0f60VgECHSMnFexO4UH6NqaOKFv-EiAJa%26wd%3D%26eqid%3Da0c9729c000721180000000268707ebb%22%5D; _pk_ses.100001.3ac3=1; dbcl2="208442265:AfTDHM67Jc8"; ck=6QfM',
    }

    # 构造参数字典
    params = {
        "start": page * 20,
        "limit": limit,
        "status": review_type,
        "sort": "score"
    }

    # 构造请求 URL（基础 URL）
    url = f"https://book.douban.com/subject/{book_id}/comments/"

    # 发送 GET 请求
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.text
    else:
        print(f"请求失败，状态码: {response.status_code}")

In [98]:
def write_csv_headers(file_path: str, headers: list):
    import csv
    with open(file_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(headers)
    

def save_data(data: dict, file_path: str):
    """
    保存数据到文件
    :param data:
    :param file_path:
    :return:
    """

    import csv
    with open(file_path, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(data.values())

In [99]:
from bs4 import BeautifulSoup
# 解析网页组件
def parse_html(html: str, review_type: str , file_path: str, book_id: str):
    # 解析 HTML 文档
    soup = BeautifulSoup(html, "html.parser")
    # 找到class为comment-list score的div标签
    comments = soup.find_all("div", class_="comment-list score")
    # 遍历评论列表
    for comment in comments:
        # 找到class为comment-item的li标签
        items = comment.find_all("li", class_="comment-item")
        # 遍历评论项
        for item in items:
            try:
                # 被认为有用数vote-count的span标签内容：评论的点赞数
                vote_count = item.find("span", class_="vote-count").text.strip()
                # comment-info中的第一个a标签的内容：评论者名字
                author = item.find("span", class_="comment-info").find_all("a")[0].text.strip()
                # comment-info中的第二个a标签的内容：评论时间
                time = item.find("span", class_="comment-info").find_all("a")[1].text.strip()
                # 类名为comment-location的span标签内容：评论所在位置
                location = item.find("span", class_="comment-location").text.strip()
                # 评论内容
                content = item.find("span", class_="short").text.strip()
            except:
                continue
            # print(f"点赞数：{vote_count}")
            # 保存为字典
            data = {
                "点赞数": vote_count,
                "评论者": author,
                "评论时间": time,
                "评论位置": location,
                "评论内容": content,
                "评论类型": review_type,
            }
            # 调用保存函数
            save_data(data, file_path + f"/{book_id}.csv")
    return len(comments[0].find_all("li", class_="comment-item"))

In [100]:
book_to_id = {
    # "伤痕": ["3013597"],
    # "班主任": ["5348123"],
    "铺花的歧路": ["19966306"],
    "许茂和他的女儿们": ["26360952", "31743822", "4144399", "1200841", "1029871", "34820949", "25981438"],
    "从森林里来的孩子": ["3640513"],
    "将军吟": ["31793467", "26354971", "1084075", "3533433", "1545838", "25981450", "1685345"],
    "灵与肉": ["1921338", "11586912", "11441802", "24714648"]
}   

In [101]:

import time
import random
import os

for title, id_list in book_to_id.items():    
    for book_id in id_list:
        path = f"./data/douban/{title}"
        os.makedirs(path, exist_ok=True)
        write_csv_headers(path + f"/{book_id}.csv", ["点赞数", "评论者", "评论时间", "评论位置", "评论内容", "评论类型"])
        print(f"开始爬取书籍：{title}，ID：{book_id}")
        for review_type in ["P", "N", "F"]:
            page = 0
            print(f"开始爬取评论类型：{review_type}", end=" ")
            while True:
                html = get_html(page, 20, book_id, review_type)
                if not html:
                    break
                comment_num = parse_html(html, review_type, path, book_id)
                page += 1
                # print(f"第{page}页爬取完成！")
                print(f"{page}",end=" ")
                time.sleep(random.randint(5,10))
                if comment_num <= 1:
                    break
        print(f"书籍：{title}，ID：{book_id} 爬取完成！")


开始爬取书籍：铺花的歧路，ID：19966306
开始爬取评论类型：P 

1 2 开始爬取评论类型：N 1 开始爬取评论类型：F 1 2 书籍：铺花的歧路，ID：19966306 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：26360952
开始爬取评论类型：P 1 2 3 4 开始爬取评论类型：N 1 2 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：26360952 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：31743822
开始爬取评论类型：P 1 2 3 4 5 6 7 8 9 10 11 12 13 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：31743822 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：4144399
开始爬取评论类型：P 1 2 3 4 5 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：4144399 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：1200841
开始爬取评论类型：P 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 开始爬取评论类型：N 1 2 开始爬取评论类型：F 1 2 书籍：许茂和他的女儿们，ID：1200841 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：1029871
开始爬取评论类型：P 1 2 3 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：1029871 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：34820949
开始爬取评论类型：P 1 2 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：34820949 爬取完成！
开始爬取书籍：许茂和他的女儿们，ID：25981438
开始爬取评论类型：P 1 2 3 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：许茂和他的女儿们，ID：25981438 爬取完成！
开始爬取书籍：从森林里来的孩子，ID：3640513
开始爬取评论类型：P 1 2 开始爬取评论类型：N 1 开始爬取评论类型：F 1 书籍：从森林里来的孩子，ID：3640513 爬取完成！
开始爬取书籍：将军吟，ID：31793467
开始爬取评论类型：P 1 2 3 4 5 6 7 开始爬取评论类型：N 1 

In [ ]:
# create dictionary for book to book_id list

book_to_id = {
    "伤痕": ["3013597"],
    "班主任": ["5348123"],
    "铺花的歧路": ["19966306"],
    "许茂和他的女儿们": ["26360952", "31743822", "4144399", "1200841", "1029871", "34820949", "25981438"],
    "从森林里来的孩子": ["3640513"],
    "将军吟": ["31793467", "26354971", "1084075", "3533433", "1545838", "25981450", "1685345"],
    "灵与肉": ["1921338", "11586912", "11441802", "24714648"]
}   

In [63]:
import pandas as pd

In [67]:
df = pd.read_csv("./data/douban/伤痕/3013597.csv", encoding="utf-8")

In [68]:
len(df)

280

In [69]:
df.tail(10)

,78,乌有,2012-03-18 18:20:23,Unnamed: 3,文学史价值大于文学价值,P
270,1,momo,2019-12-16 08:46:24,NaN,结尾给我看愣了，被qj了还得笑着说谢谢的感觉,P
271,0,刀刀,2018-12-18 18:50:26,NaN,3.5\n现在都有点害怕在把政治映射到人物身上，有一种在背后操纵的感觉，像提线木偶。,P
272,1,GH,2017-10-21 23:56:53,NaN,据说是伤痕文学的发轫之作。校友写的，哈哈。感觉有点刻意吧，比如主人公回去刚好赶上妈妈去世。当...,P
273,2,灵活的金鱼,2020-04-20 20:05:19,NaN,短短的几千字，我却哭了两次，有悲痛也有淡淡的温情。是时代的使然，在她们心灵上划下来无尽的伤痕...,P
274,0,豆友QRkPq5mtJM,2024-10-07 16:45:00,北京,垃圾，故事情节简单而刻意，文学价值丝毫没有，当时能有反响纯粹是人们需要一时的新奇感，然而终究...,P
275,0,别管帅哥的事,2024-05-26 13:42:38,广东,记住该记住的，忘记该忘记的。改变能改变的，接受不能改变的。不管生活怎样，总有一道属于自己的明...,P
276,0,Winky,2025-04-15 22:06:06,广东,这个故事主要写了王晓华的母亲在“文革”期间遭到四人帮的迫害，被诬陷是“叛徒”，王晓华由于“叛...,P
277,0,不是山谷,2025-04-23 22:50:37,广东,还是想说 在成为什么身份之前 你首先是人,P
278,0,山海不說話,2025-05-15 20:19:12,辽宁,如果作者是在讽刺女主这种人，我会觉得写得还不错。,P
279,0,不是山谷,2025-04-23 22:50:37,广东,还是想说 在成为什么身份之前 你首先是人,P
